In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.precision', 15)  # Increase decimal precision
pd.set_option('display.width', 150)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

# Phương pháp Dây cung (Secant Method)

# Điều kiện:

* $(a,b)$ là khoảng cách ly nghiệm

* $f'(x)$ xác định dấu không đổi trên $[a,b]$

* $f''(x)$ xác định dấu không đổi trên $[a,b]$

Điều kiện hội tụ:

* Chọn đúng điểm mốc $d$ (điểm Fourier - $f(d) \cdot f''(d) > 0$)
    
* Chọn đúng xuất phát ban đầu $x_0$ ($f(d) \cdot f(x_0) < 0$)

# Thuật toán:

1. Kiểm tra input ban đầu có thỏa mãn điều kiện không

2. Xác định điểm mốc $d$ và điểm xuất phát ban đầu $x_0$

3. Xác định x theo công thức lặp: 

$\quad x_{n+1} = x_n - \dfrac{f(x_n) \cdot (x_n - d)}{f(x_n) - f(d)} = \dfrac{d \cdot f(x_n) - x_n \cdot f(d)}{f(x_n) - f(d)}$

4. Lặp lại bước (3) cho đến khi:

   a. $x_n$ là nghiệm của phương trình, hoặc

   b. khoảng $[d, x_n]$ (hoặc $[x_n, d]$) thỏa mãn điều kiện dừng

5. In giá trị $x_n$ (giá trị $x$ sau $n$ lần lặp)

# Áp dụng

## 1. Hậu nghiêm - Số lần lặp

Đánh giá sai số theo 1 trong 2 công thức:
+ $|x_n - x^*| \leqslant \dfrac{|f(x_n)|}{m_1}$ $\quad\quad$(1)

+ $|x_n - x^*| \leqslant \dfrac{M_1 - m_1}{m_1} \cdot |x_n - x_{n-1}| $ $\quad\quad$(2)

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_1 = \max\limits_{[a,b]} |f'(x)|$

### 1.1. Công thức 1



In [3]:
def secant_iteration_v1 (f, df, a, b, n, rbl):	
	M1 = max([np.abs(df(x)) for x in [a, b]]) #M1 is the maximum value of |f'(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]

	# Error function
	if (f(a) * f(b) >= 0) or (M1 * m1 <= 0): 
		print("Can not find a root in the given interval");
		return 0;

	# Implementing Secant Method
	x = a; mrk = b;

	results = [];
	for i in range(n):
		# Calculate the next value of x
		x_new = (mrk * f(x) - x * f(mrk)) / (f(x) - f(mrk))
		delta_x = abs(f(x_new))/ m1

		results.append({
            'n': i,
			'mrk': mrk,
            'x_(n+1)': x_new,
            'f(x_(n+1))': f(x_new),
            'delta_x=|f(x_n)| / m_1': delta_x
        })
		
		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break
		elif (i == 0):
			if f(a) * f(x_new) < 0:
				mrk = a
			else:
				mrk = b

	# Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))

	if rbl == None:
		print(f"The value of root is: {x}")
	else:
		total_delta = delta_x + 0.5 * 10**(-rbl) #must calculate roundoff error
		print(f"The value of root with {rbl} decimal point is: {round(x, rbl)}")
		print(f"Relative error is: {total_delta}")

In [4]:
f = lambda x: x ** 5 - 12; 
df = lambda x: 5 * (x ** 4)

a = 1
b = 2

n = 20
rbl = 9;
secant_iteration_v1 (f, df, a, b, n, rbl)

 n  mrk           x_(n+1)         f(x_(n+1))  delta_x=|f(x_n)| / m_1
 0    2 1.354838709677419 -7.435029421585015       1.487005884317003
 1    2 1.529680628069611 -3.624633240286681       0.724926648057336
 2    2 1.601839853218664 -1.453812878149883       0.290762575629977
 3    2 1.628821087381766 -0.535188563101631       0.107037712620326
 4    2 1.638494760856317 -0.190669278217582       0.038133855643516
 5    2 1.641908612178906 -0.067130140804897       0.013426028160979
 6    2 1.643106527631529 -0.023536232519628       0.004707246503926
 7    2 1.643526030343380 -0.008239825857270       0.001647965171454
 8    2 1.643672834033169 -0.002883205004572       0.000576641000914
 9    2 1.643724194842391 -0.001008683129713       0.000201736625943
10    2 1.643742162405922 -0.000352863397884       0.000070572679577
11    2 1.643748447812582 -0.000123438003291       0.000024687600658
12    2 1.643750646548026 -0.000043180514584       0.000008636102917
13    2 1.643751415697885 -0.00001

### 1.2. Công thức 2

In [5]:
def secant_iteration_v2 (f, df, a, b, n, rbl):	
	M1 = max([np.abs(df(x)) for x in [a, b]]) #M1 is the maximum value of |f'(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]

	# Error function
	if (f(a) * f(b) >= 0) or (M1 * m1 <= 0): 
		print("Can not find a root in the given interval");
		return 0;

	# Implementing Secant Method
	x = a; mrk = b;

	results = [];
	for i in range(n):
		# Calculate the next value of x
		x_new = (mrk * f(x) - x * f(mrk)) / (f(x) - f(mrk))
		delta_x = (M1 - m1) * abs(x - x_new) / m1

		results.append({
            'n': i,
			'mrk': mrk,
            'x_(n+1)': x_new,
            'f(x_(n+1))': f(x_new),
            'delta_x=(M1 - m1)*(x-x_new)/m1': delta_x
        })
		
		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break
		elif (i == 0):
			if f(a) * f(x_new) < 0:
				mrk = a
			else:
				mrk = b

	# Print the final result
	df_results = pd.DataFrame(results)
	print(df_results.to_string(index=False))

	if rbl == None:
		print(f"The value of root is: {x}")
	else:
		total_delta = delta_x + 0.5 * 10**(-rbl) #must calculate roundoff error
		print(f"The value of root with {rbl} decimal point is: {round(x, rbl)}")
		print(f"Relative error is: {total_delta}")

In [6]:
f = lambda x: x ** 5 - 12; 
df = lambda x: 5 * (x ** 4)

a = 1
b = 2

n = 20
rbl = 9;
secant_iteration_v2 (f, df, a, b, n, rbl)

 n  mrk           x_(n+1)         f(x_(n+1))  delta_x=(M1 - m1)*(x-x_new)/m1
 0    2 1.354838709677419 -7.435029421585015               5.322580645161289
 1    2 1.529680628069611 -3.624633240286681               2.622628775882871
 2    2 1.601839853218664 -1.453812878149883               1.082388377235792
 3    2 1.628821087381766 -0.535188563101631               0.404718512446542
 4    2 1.638494760856317 -0.190669278217582               0.145105102118266
 5    2 1.641908612178906 -0.067130140804897               0.051207769838835
 6    2 1.643106527631529 -0.023536232519628               0.017968731789334
 7    2 1.643526030343380 -0.008239825857270               0.006292540677775
 8    2 1.643672834033169 -0.002883205004572               0.002202055346837
 9    2 1.643724194842391 -0.001008683129713               0.000770412138328
10    2 1.643742162405922 -0.000352863397884               0.000269513452962
11    2 1.643748447812582 -0.000123438003291               0.000094281099904

## 2. Hậu nghiệm - Sai số tuyệt đối

Đánh giá sai số theo 1 trong 2 công thức:
+ $|x_n - x^*| \leqslant \dfrac{|f(x_n)|}{m_1} < \epsilon$ $\quad\quad(1)$

+ $|x_n - x^*| \leqslant \dfrac{M_1 - m_1}{m_1} \cdot |x_n - x_{n-1}| < \epsilon$ $\quad\quad(2)$

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_1 = \max\limits_{[a,b]} |f'(x)|$

### 2.1. Công thức 1

Từ công thức (1) suy ra Điều kiện dừng: $|f(x_n)| < m_1 \epsilon $

In [7]:
def secant_recursion_absolute_v1 (f, df, a, b, eps):	
	M1 = max([np.abs(df(x)) for x in [a, b]]) #M1 is the maximum value of |f'(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]

	# Error function
	if (f(a) * f(b) >= 0) or (M1 * m1 <= 0): 
		print("Can not find a root in the given interval");
		return 0;

	# Implementing Secant Method
	x = a; mrk = b; new_eps = m1*eps;
	print(f"delta_x = {new_eps}")

	results = []; i = 0; 
	while True:
		# Calculate the next value of x
		x_new = (mrk * f(x) - x * f(mrk)) / (f(x) - f(mrk))
		current_delta_x = abs(f(x_new))

		results.append({
            'n': i,
			'mrk': mrk,
            'x_(n+1)': x_new,
            'f(x_(n+1))': f(x_new),
            'delta_x=|f(x_n)|': current_delta_x
        })
		
		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break
		elif (i == 0):
			if f(a) * f(x_new) < 0:
				mrk = a
			else:
				mrk = b
				
        # Stopping condition
		if current_delta_x < new_eps:
			break
		else:
			i += 1

    # Print the final result
	df_results = pd.DataFrame(results) 
	print(df_results.to_string(index=False))

	print(f"The value of root with absolute error {eps} is: {x}")

In [8]:
f = lambda x: np.log(x) - 1 #approximate e
df = lambda x: 1/x

a = 2
b = 3

eps = 0.5 * pow(10, -7)

secant_recursion_absolute_v1 (f, df, a, b, eps)

delta_x = 1.6666666666666664e-08
 n  mrk           x_(n+1)        f(x_(n+1))  delta_x=|f(x_n)|
 0    3 2.756792171024977 0.014067746909731 0.014067746909731
 1    2 2.723617728992985 0.001961044002238 0.001961044002238
 2    2 2.719022578401181 0.000272469551561 0.000272469551561
 3    2 2.718384689674588 0.000037839810558 0.000037839810558
 4    2 2.718296112392272 0.000005254751567 0.000005254751567
 5    2 2.718283812023015 0.000000729712138 0.000000729712138
 6    2 2.718282103910405 0.000000101332887 0.000000101332887
 7    2 2.718281866710123 0.000000014071785 0.000000014071785
The value of root with absolute error 5e-08 is: 2.718281866710123


In [9]:
f = lambda x: np.tan(x/4) - 1 #approximate pi
df = lambda x: (1/4) * 1/(np.cos(x/4)**2)

a = 3
b = 4

eps = 0.5 * pow(10, -7)

secant_recursion_absolute_v1 (f, df, a, b, eps)

delta_x = 2.3348399552254093e-08
 n  mrk           x_(n+1)         f(x_(n+1))  delta_x=|f(x_n)|
 0    4 3.109303785203571 -0.016015501450374 0.016015501450374
 1    4 3.134180605448197 -0.003699173691322 0.003699173691322
 2    4 3.139888637794978 -0.000851645144732 0.000851645144732
 3    4 3.141200769225595 -0.000195922987937 0.000195922987937
 4    4 3.141502522114125 -0.000045064722404 0.000045064722404
 5    4 3.141571923418169 -0.000010365032095 0.000010365032095
 6    4 3.141587885646036 -0.000002383969037 0.000002383969037
 7    4 3.141591556960693 -0.000000548314400 0.000000548314400
 8    4 3.141592401364576 -0.000000126112601 0.000000126112601
 9    4 3.141592595577870 -0.000000029005961 0.000000029005961
10    4 3.141592640247022 -0.000000006671385 0.000000006671385
The value of root with absolute error 5e-08 is: 3.1415926402470222


### 2.2. Công thức 2

Từ công thức (2) suy ra Điều kiện dừng: $|x_n - x_{n-1}| < \epsilon \dfrac{m_1}{M_1 - m_1}$

In [10]:
def secant_recursion_absolute_v2 (f, df, a, b, eps):	
	M1 = max([np.abs(df(x)) for x in [a, b]]) #M1 is the maximum value of |f'(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]

	# Error function
	if (f(a) * f(b) >= 0) or (M1 * m1 <= 0): 
		print("Can not find a root in the given interval");
		return 0;

	# Implementing Secant Method
	x = a; mrk = b; new_eps = eps*m1/(M1 - m1);
	print(f"delta_x = {new_eps}")

	results = []; i = 0; 
	while True:
		# Calculate the next value of x
		x_new = (mrk * f(x) - x * f(mrk)) / (f(x) - f(mrk))
		current_delta_x = abs(x - x_new)

		results.append({
            'n': i,
			'mrk': mrk,
            'x_(n+1)': x_new,
            'f(x_(n+1))': f(x_new),
            'delta_x=|x_(n+1) - x_n|': current_delta_x
        })
		
		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break
		elif (i == 0):
			if f(a) * f(x_new) < 0:
				mrk = a
			else:
				mrk = b
				
        # Stopping condition
		if current_delta_x < new_eps:
			break
		else:
			i += 1

    # Print the final result
	df_results = pd.DataFrame(results) 
	print(df_results.to_string(index=False))

	print(f"The value of root with absoulte error {eps} is: {x}")

In [11]:
f = lambda x: x**5 - 17
df = lambda x: 5 * x**4

a = 1
b = 2

eps = 0.5 * pow(10, -6)

secant_recursion_absolute_v2 (f, df, a, b, eps)

delta_x = 3.333333333333333e-08
 n  mrk           x_(n+1)         f(x_(n+1))  delta_x=|x_(n+1) - x_n|
 0    2 1.516129032258065 -8.989109037847472        0.516129032258065
 1    2 1.697443347951021 -2.907876413733710        0.181314315692957
 2    2 1.746572420096993 -0.747020411714363        0.049129072145971
 3    2 1.758594730993223 -0.179889751924819        0.012022310896231
 4    2 1.761455511582849 -0.042633986796407        0.002860780589626
 5    2 1.762131596806917 -0.010065952429237        0.000676085224068
 6    2 1.762291114562438 -0.002374452888258        0.000159517755521
 7    2 1.762328737176869 -0.000559989851265        0.000037622614431
 8    2 1.762337609745307 -0.000132061138423        0.000008872568438
 9    2 1.762339702124607 -0.000031143311904        0.000002092379300
10    2 1.762340195558835 -0.000007344349665        0.000000493434228
11    2 1.762340311922558 -0.000001731974969        0.000000116363723
12    2 1.762340339363931 -0.000000408441448        0.0000

In [12]:
f = lambda x: np.e**x - np.cos(2*x)
df = lambda x: np.e**x + 2*np.sin(2*x)

a = -0.5
b = -0.1

eps = 0.5 * pow(10, -8)

secant_recursion_absolute_v2 (f, df, a, b, eps)

delta_x = 4.460252752041862e-09
 n  mrk            x_(n+1)         f(x_(n+1))  delta_x=|x_(n+1) - x_n|
 0 -0.1 -0.312725808235359 -0.079248354484380        0.187274191764641
 1 -0.5 -0.414743318153092 -0.014744698476805        0.102017509917733
 2 -0.5 -0.430268039414401 -0.001696355068702        0.015524721261309
 3 -0.5 -0.432009528876521 -0.000183203510066        0.001741489462120
 4 -0.5 -0.432197087992088 -0.000019648493918        0.000187559115567
 5 -0.5 -0.432217197656640 -0.000002105717064        0.000020109664552
 6 -0.5 -0.432219352728558 -0.000000225650322        0.000002155071918
 7 -0.5 -0.432219583666999 -0.000000024180661        0.000000230938441
 8 -0.5 -0.432219608414325 -0.000000002591194        0.000000024747326
 9 -0.5 -0.432219611066243 -0.000000000277672        0.000000002651918
The value of root with absoulte error 5e-09 is: -0.4322196110662426


## 3. Hậu nghiệm - Sai số tương đối:

Đánh giá sai số theo công thức: $\dfrac{|x_n - x^*|}{|x_n|} \leqslant \dfrac{M_1 - m_1}{m_1} \cdot \dfrac{|x_n - x_{n-1}|}{|x_n|} < \eta$

với $m_1 = \min\limits_{[a,b]} |f'(x)|, \quad M_1 = \max\limits_{[a,b]} |f'(x)|$

### 3.1. Công thức mục tiêu:

Từ công thức sai số hậu nghiệm, ta có Điều kiện dừng:  $\dfrac{|x_n - x_{n-1}|}{|x_n|} < \eta \dfrac{m_1}{M_1 - m_1}$

In [20]:
def secant_recursion_relative (f, df, a, b, eta):	
	M1 = max([np.abs(df(x)) for x in [a, b]]) #M1 is the maximum value of |f'(x)| in the interval [a,b]
	m1 = min([np.abs(df(x)) for x in [a, b]]) #m1 is the minimum value of |f'(x)| in the interval [a,b]

	# Error function
	if (f(a) * f(b) >= 0) or (M1 * m1 <= 0): 
		print("Can not find a root in the given interval");
		return 0;

	# Implementing Secant Method
	x = a; mrk = b; new_eta = eta * m1/(M1 - m1);
	print(f"sigma_x = {new_eta}, m_1 = {m1}, M_1 = {M1}")

	results = []; i = 0;
	while True:
		# Calculate the next value of x
		x_new = (mrk * f(x) - x * f(mrk)) / (f(x) - f(mrk))
		current_sigma_x = abs(x - x_new)/abs(x_new)

		results.append({
            'n': i,
			'mrk': mrk,
            'x_(n+1)': x_new,
            'f(x_(n+1))': f(x_new),
            'sigma_x=|x_(n+1)-x_n|/|x_n|': current_sigma_x
        })
		
		# Prepare for next iteration
		x = x_new
		if (f(x_new) == 0): 
			break
		elif (i == 0):
			if f(a) * f(x_new) < 0:
				mrk = a
			else:
				mrk = b
				
        # Stopping condition
		if current_sigma_x < new_eta:
			break
		else:
			i += 1

    # Print the final result
	df_results = pd.DataFrame(results) 
	print(df_results.to_string(index=False))

	print(f"The value of root with relative error {eta} is: {x}")

In [14]:
f = lambda x: x**5 - 3*x**3 + 2*x**2 - x + 5
df = lambda x: 5*x**4 - 9*x**2 + 4*x - 1

a = -3
b = -2

eta = 0.05 * pow(10, -2)

secant_recursion_relative (f, df, a, b, eta)

sigma_x = 6.340579710144928e-05
 n  mrk            x_(n+1)        f(x_(n+1))  sigma_x=|x_(n+1)-x_n|/|x_n|
 0   -2 -2.048951048951049 5.138543296012815            0.464163822525597
 1   -3 -2.083576645173499 3.633621955959093            0.016618345335488
 2   -3 -2.107424311490581 2.500536154289734            0.011316025058197
 3   -3 -2.123539178923813 1.688293466358496            0.007588683831771
 4   -3 -2.134286084419477 1.125129162756630            0.005035363147479
 5   -3 -2.141389377440757 0.743280711796173            0.003317142176996
 6   -3 -2.146056434654608 0.488176034016396            0.002174713180179
 7   -3 -2.149110726939240 0.319399617813440            0.001421188888198
 8   -3 -2.151104380882693 0.208449064921011            0.000926804836237
 9   -3 -2.152403503655438 0.135816173304101            0.000603568415745
10   -3 -2.153249110020282 0.088396961042743            0.000392711814397
11   -3 -2.153799121682598 0.057493650009568            0.000255368133815
12   -

In [15]:
f = lambda x: np.e**(-x) - x
df = lambda x: -np.e**(-x) - 1

a = 0
b = 1

eta = 0.05 * pow(10, -2)

secant_recursion_relative (f, df, a, b, eta)

sigma_x = 0.0010819767068693264
 n  mrk           x_(n+1)         f(x_(n+1))  sigma_x=|x_(n+1)-x_n|/|x_n|
 0    1 0.612699836780282 -0.070813947873171            1.000000000000000
 1    0 0.572181412090508 -0.007888272855300            0.070813947873171
 2    0 0.567703214235785 -0.000877391979772            0.007888272855300
 3    0 0.567205552633023 -0.000097572726128            0.000877391979772
The value of root with relative error 0.0005 is: 0.5672055526330225


In [21]:
f = lambda x: 2**x - 5*x + np.sin(x)
df = lambda x: 2**x*np.log(2) - 5 + np.cos(x)

a = 0
b = 1

eta = 0.05 * pow(10, -2)

secant_recursion_relative (f, df, a, b, eta)

sigma_x = 0.006582587479708893, m_1 = 3.0734033330119694, M_1 = 3.3068528194400546
 n  mrk           x_(n+1)         f(x_(n+1))  sigma_x=|x_(n+1)-x_n|/|x_n|
 0    1 0.316603075415845 -0.026280379774505            1.000000000000000
 1    0 0.308495691484825 -0.000437047622463            0.026280379774506
 2    0 0.308360923076534 -0.000007294904439            0.000437047622463
The value of root with relative error 0.0005 is: 0.308360923076534
